In [1]:
import os
import boto3  # AWS
import pandas as pd
from pandas import json_normalize
import numpy as np
import json
import io
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from dash_functions import find_closest_poi


# Set the maximum number of rows to 200
pd.set_option("display.max_rows", None) 

In [2]:
# Load AWS Files
s3R = boto3.resource("s3")
s3 = boto3.client("s3")
bucket_name = "capstonehaystacks"

# Listing the contents of the S3 bucket to ensure everything looks good
for obj in s3R.Bucket(bucket_name).objects.all():
    print(obj.key)

GA_LISTINGS_SALES.csv
GA_LISTINGS_SALES_V2.csv
GA_listing.csv
all_zips_grocery_store.json
all_zips_restaurant.json
atlanta-geo.csv
atlanta_cbsa_zip.csv
atlanta_hdma_2021.csv
atlanta_hdma_census_2021.csv
atlanta_listings.csv
core_geo_dataset.csv
crime.csv
crime_rating_zipcode.csv
elementary_schools.csv
ga_georgia_zip_codes_geo.min.json
high_schools.csv
hmda_2021_ga_all-records_labels.csv
jason_listing.csv
listings_with_tract.csv
middle_schools.csv
poi_combined_haystack.csv
poi_combined_haystack_ALL.csv
poi_combined_haystack_ALL_CLEANED.csv
points-of-interest-google.csv
points-of-interest-google2.csv
points-of-interest-haystacks.csv
schools.csv
zip_summary.csv
zipcodes_111meter.min.json


In [ ]:
atlanta

In [3]:
file_name = "poi_combined_haystack_ALL_CLEANED.csv"

file_obj = s3.get_object(Bucket=bucket_name, Key=file_name)
POI = pd.read_csv(file_obj["Body"])
print(POI.shape)
POI.head(3)

(13331, 10)


,latitude,longitude,state,place_id,name,rating,num_ratings,poi_types,zipcode,address
0,33.791987,-84.385677,GA,ChIJoz4--E8E9YgRxpSO8xC1jj0,Residence Inn Atlanta Midtown/Peachtree at 17th,4.1,723.0,lodging,30309,"1365 Peachtree Street Northeast, Atlanta"
1,33.790216,-84.388776,GA,ChIJh-lI8FoE9YgRyKGKYvLpvA8,Artmore Hotel,3.9,1106.0,lodging,30309,"1302 West Peachtree Street Northwest, Atlanta"
2,33.799427,-84.392589,GA,ChIJFybLXVQE9YgRTePNqN08VnU,Atlanta Peachtree Station,3.6,163.0,train_station,30309,"1688 Peachtree Road Northwest, Atlanta"


In [14]:
address = '1091 S Jeff Davis Dr, Fayetteville, GA 30215' 
df = POI
cats = ['school','primary_School','secondary_school']
school_indicator = find_closest_poi(address, df, cats)
school_indicator

{'latitude': 33.3873535,
 'longitude': -84.4747677,
 'name': 'Whitewater Middle School',
 'address': '1533 Georgia 85, Fayetteville',
 'distance_miles': 4.53}

In [15]:
school_indicator['distance_miles']

4.53

In [26]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    mode = "number",
    value = school_indicator['distance_miles'],
    number = {'suffix': " mi"},
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': f"Distance to Nearest School<br><span style='font-size:0.8em;color:gray'>{school_indicator['name']}</span>"}))

fig.update_layout(paper_bgcolor = "lightgray")

fig.show()

In [22]:
POI.columns

Index(['latitude', 'longitude', 'state', 'place_id', 'name', 'rating',
       'num_ratings', 'poi_types', 'zipcode', 'address'],
      dtype='object')

In [29]:
def worship_indicator(user_address):
    cats = ['hindu_temple', 'place_of_worship', 'church']
    closest = find_closest_poi(listing_address=user_address, poi_dataframe=POI, poi_categories=cats)

    fig = go.Figure(go.Indicator(
        mode="number",
        value=closest['distance_miles'],
        number={'suffix': " mi"},
        domain={'x': [0, 1], 'y': [0, 1]},
        title = {'text': f"Distance to Nearest Place of Worship<br><span style='font-size:0.8em;color:gray'>{closest['name']}</span>"}))

    fig.update_layout(paper_bgcolor="lightgray")

    return fig

In [30]:
worship_indicator(user_address = '2120 Blayton Ln SE, Atlanta, GA 30315')

In [43]:
def restaurant_indicator(user_address):
    cats = ['restaurant', 'meal_takeaway']
    closest = find_closest_poi(listing_address=user_address, poi_dataframe=POI, poi_categories=cats)

    fig = go.Figure(go.Indicator(
        mode="number",
        value=closest['distance_miles'],
        number={'suffix': " mi"},
        domain={'x': [0, 1], 'y': [0, 1]},
        title = {'text': f"Distance to Nearest Restaurant<br><span style='font-size:0.8em;color:gray'>{closest['name']}</span>"}))

    fig.update_layout(paper_bgcolor="lightgray")

    return fig

In [44]:
restaurant_indicator(user_address = '2120 Blayton Ln SE, Atlanta, GA 30315')

In [41]:
def grocery_indicator(user_address):
    cats = ['convenience_store', 'grocery_or_supermarket', 'supermarket']
    closest = find_closest_poi(listing_address=user_address, poi_dataframe=POI, poi_categories=cats)

    fig = go.Figure(go.Indicator(
        mode="number",
        value=closest['distance_miles'],
        number={'suffix': " mi"},
        domain={'x': [0, 1], 'y': [0, 1]},
        title = {'text': f"Distance to Nearest Grocery<br><span style='font-size:0.8em;color:gray'>{closest['name']}</span>"}))

    fig.update_layout(paper_bgcolor="lightgray")

    return fig

In [42]:
grocery_indicator(user_address = '2120 Blayton Ln SE, Atlanta, GA 30315')

In [45]:
def airport_indicator(user_address):
    cats = ['airport']
    closest = find_closest_poi(listing_address=user_address, poi_dataframe=POI, poi_categories=cats)

    fig = go.Figure(go.Indicator(
        mode="number",
        value=closest['distance_miles'],
        number={'suffix': " mi"},
        domain={'x': [0, 1], 'y': [0, 1]},
        title = {'text': f"Distance to Nearest Airport<br><span style='font-size:0.8em;color:gray'>{closest['name']}</span>"}))

    fig.update_layout(paper_bgcolor="lightgray")

    return fig

In [47]:
airport_indicator(user_address = '2120 Blayton Ln SE, Atlanta, GA 30315')

In [33]:
POI.poi_types.value_counts()

food                       3207
restaurant                 2956
store                      1011
meal_takeaway               787
bar                         586
health                      535
lodging                     315
finance                     227
home_goods_store            199
doctor                      196
convenience_store           195
cafe                        192
school                      182
meal_delivery               174
place_of_worship            136
church                      134
clothing_store              125
gas_station                 115
local_government_office     114
atm                         106
general_contractor          104
bakery                       97
grocery_or_supermarket       94
secondary_school             84
drugstore                    84
furniture_store              79
car_repair                   75
electronics_store            61
real_estate_agency           54
department_store             50
tourist_attraction           46
pharmacy

In [48]:
address = '1091 S Jeff Davis Dr, Fayetteville, GA 30215' 
df = POI
cats = ['school','primary_School','secondary_school']
school_indicator = find_closest_poi(address, df, cats)
school_indicator

{'latitude': 33.3873535,
 'longitude': -84.4747677,
 'name': 'Whitewater Middle School',
 'address': '1533 Georgia 85, Fayetteville',
 'distance_miles': 4.53}

In [50]:
list_of_cats = {'airport':['airport'],
'school':['school', 'primary_School', 'secondary_school'],
'worship':['hindu_temple', 'place_of_worship', 'church'],
'grocery':['convenience_store', 'grocery_or_supermarket', 'supermarket'],
'restaurant':['restaurant', 'meal_takeaway']}
type(list_of_cats)

dict

In [51]:
list_of_cats['school']

['school', 'primary_School', 'secondary_school']

In [52]:
import pandas as pd

list_of_cats = {
    'airport': ['airport'],
    'school': ['school', 'primary_school', 'secondary_school'],
    'worship': ['hindu_temple', 'place_of_worship', 'church'],
    'grocery': ['convenience_store', 'grocery_or_supermarket', 'supermarket'],
    'restaurant': ['restaurant', 'meal_takeaway']
}

address = '2120 Blayton Ln SE, Atlanta, GA 30315'
result_df = pd.DataFrame(columns=['Category', 'Name', 'Distance (miles)', 'Latitude', 'Longitude'])

for category, poi_categories in list_of_cats.items():
    closest_poi = find_closest_poi(address, POI, poi_categories)
    result_df = result_df.append({
        'Category': category,
        'Name': closest_poi['name'],
        'Distance (miles)': closest_poi['distance_miles'],
        'Latitude': closest_poi['latitude'],
        'Longitude': closest_poi['longitude']
    }, ignore_index=True)

result_df


/var/folders/_j/zgrrpz8955l530bsdd3qxdq80000gp/T/ipykernel_72705/2043203318.py:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/_j/zgrrpz8955l530bsdd3qxdq80000gp/T/ipykernel_72705/2043203318.py:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/_j/zgrrpz8955l530bsdd3qxdq80000gp/T/ipykernel_72705/2043203318.py:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/_j/zgrrpz8955l530bsdd3qxdq80000gp/T/ipykernel_72705/2043203318.py:16: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/_j/zgrrpz8955l530bsdd3qxdq80000gp/T/ipykernel_72705/2043203318.py:16: FutureWarning:

The frame.append method is de

,Category,Name,Distance (miles),Latitude,Longitude
0,airport,Hartsfield-Jackson Atlanta International Airport,5.3,33.640728,-84.4277
1,school,Ronald McNair Sr. High School,2.27,33.704861,-84.326301
2,worship,Episcopal Church of the Incarnation,6.15,33.721878,-84.466738
3,grocery,ALDI,1.44,33.713723,-84.350202
4,restaurant,Sonic Drive-In,1.42,33.713302,-84.350159
